In [11]:
%%time
# core importations
from caveclient import CAVEclient
import pandas as pd

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

# ----------CORE DATA GATHERING FUCNTIONS----------

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to get nucleus table entry using root id
def getNucRoot(root_ids):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": root_ids},
                                            materialization_version = mat_vers)
    return nuc_df 

def getNucNuc(nucleus_ids):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"id": nucleus_ids},
                                            materialization_version = mat_vers)
    return nuc_df

# defines function to get presynaptic table entry using root id
def getPreSyn(root_id):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    pre_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root_id]})
    return len(pre_syn_df)

# defines function to get postsynaptic table entry using root id
def getPostSyn(root_id):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    post_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"post_pt_root_id":[root_id]})
    return len(post_syn_df)

# defines function to get edit data
def getEdits(root_id):
    client = CAVEclient("flywire_fafb_production")
    changes_dict = client.chunkedgraph.get_change_log(root_id)
    return changes_dict   

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilderRoot(root_list,options):
    
    df = pd.DataFrame(root_list, columns = ['Root ID']) #creates dataframe out of root_id list#

    if 'nucs' in options:
        nuc_df = getNucRoot(root_list)
        df['Nucleus ID'] = nuc_df['id']
        df['Nucleus Coordinates'] = nuc_df['pt_position'] #adds nucleus coordinates to df#
        df['Nucleus Volume'] = nuc_df['volume']

    if 'syns' in options:
        pre_list = []
        post_list = []
        total_list = []
        for i in root_list:
            try:
                pre_list.append(getPreSyn(i))      #pulls synapse data#
            except:
                pre_list += ['n/a']
            try:
                post_list.append(getPostSyn(i))
            except:
                post_list += ['n/a']
            try:
                total_list.append(int(pre_list[-1] + post_list[-1]))
            except:
                total_list += ['n/a']
                
        df['Incoming Synapses'] = post_list
        df['Outgoing Synapses'] = pre_list
        df['Total Synapses'] = total_list
    
    if 'edits' in options: #CHANGE THIS TO TABULAR LOG#
        split_list = []
        merge_list = []
        edit_list = []
        for i in root_list:
            try:
                change_dict = getEdits(i)                         #pulls changelog#
                split_list.append(change_dict['n_splits'])        #appends split list#
                merge_list.append(change_dict['n_mergers'])       #appends merge list#
                edit_list.append(split_list[-1] + merge_list[-1]) #appends edit list#
            except:
                split_list += ['n/a']
                merge_list += ['n/a']
                edit_list += ['n/a']
        df['Splits'] = split_list
        df['Mergers'] = merge_list
        df['Edits'] = edit_list
           
    return df

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilderNuc(nuc_list,options):
    
    nuc_df = getNucNuc(nuc_list)                        #creates nuc_df using nucleus ids#
    root_list = nuc_df['pt_root_id'].to_list()           #converts pt_root_id column of nuc_df to list of root ids#
    df = pd.DataFrame(root_list, columns = ['Root ID']) #creates dataframe out of root_id list#

    if 'nucs' in options:
        df['Nucleus ID'] = nuc_list
        df['Nucleus Coordinates'] = nuc_df['pt_position']
        df['Nucleus Volume'] = nuc_df['volume']

    if 'syns' in options:
        pre_list = []
        post_list = []
        total_list = []
        for i in root_list:
            try:
                pre_list.append(getPreSyn(i))      #pulls synapse data#
            except:
                pre_list += ['n/a']
            try:
                post_list.append(getPostSyn(i))
            except:
                post_list += ['n/a']
            try:
                total_list.append(int(pre_list[-1] + post_list[-1]))
            except:
                total_list += ['n/a']
                
        df['Incoming Synapses'] = post_list
        df['Outgoing Synapses'] = pre_list
        df['Total Synapses'] = total_list
    
    if 'edits' in options: #CHANGE THIS TO TABULAR LOG#
        split_list = []
        merge_list = []
        edit_list = []
        for i in root_list:
            try:
                change_dict = getEdits(i)                         #pulls changelog#
                split_list.append(change_dict['n_splits'])        #appends split list#
                merge_list.append(change_dict['n_mergers'])       #appends merge list#
                edit_list.append(split_list[-1] + merge_list[-1]) #appends edit list#
            except:
                split_list += ['n/a']
                merge_list += ['n/a']
                edit_list += ['n/a']
        df['Splits'] = split_list
        df['Mergers'] = merge_list
        df['Edits'] = edit_list
           
    return df
   

Wall time: 0 ns


In [12]:
# DASH APP

app = dash.Dash(__name__)

# defines layout of various app elements (submission field, checkboxes, button, output table)#
app.layout = html.Div([
    html.Div(dcc.Input(  #defines input field#
        id='input_field', 
        type='text', 
        placeholder='ID Number',
    )),
    html.Br(
    ),
    dcc.Dropdown(
        id='query_type',
        options=[
            {'label': 'Root ID', 'value': 'root_query'},
            {'label': 'Nucleus ID', 'value': 'nuc_query'},
        ],
        value='root_query'
    ),
    html.Br(
    ),
    html.Div(dcc.Checklist(  #defines data selection checkboxes#
        id='checkboxes',
        options=[
            {'label': 'Nucleus Info', 'value': 'nucs'},
            {'label': 'Synapse Count', 'value': 'syns'},
            {'label': 'Edits', 'value': 'edits'},
        ],
        labelStyle={'display': 'block'},
        value=['nucs','syns','edits'],
    )),
    html.Br(
    ),
    html.Button(  #defines submission button#
        'Submit', 
        id='submit_button', 
        n_clicks=0,
    ),
    html.Br(
    ),
    html.Div(dash_table.DataTable(  #defines output table#
        id='table', 
        fill_width=False,  #sets column width to fit text instead of expanding to container width#
        export_format="csv",
    ))
])

# defines callback that takes root ids and desired data selection on button click and generates table
@app.callback(
    Output('table','columns'),           #defines first output location as the 'columns' aspect of 'table'#
    Output('table', 'data'),             #defines second output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'),  #defines trigger as button press (change in the state of the 'n_clicks' aspect of 'submit_button')# 
    State('query_type', 'value'),         #defines first input state as value of 'query_type'#
    State('input_field', 'value'),       #defines second input state as the value of 'input_field'#
    State('checkboxes','value'),         #defines third input state as the value of 'checkboxes'#
    prevent_initial_call=True,
)
def update_output(n_clicks, query_method, ids, checked):
    id_list = str(ids).split(",")                             #splits 'roots' string into list#
    id_list = [int(x.strip(' ')) for x in id_list]            #strips spaces from root_list entries and converts to integers#
    if query_method == 'root_query':                          #creates dataframe for root id queries#
        df = dfBuilderRoot(id_list, checked)
    elif query_method == 'nuc_query':                         #creates dataframe for nucleus id queries#
        df = dfBuilderNuc(id_list, checked)
    column_list = [{"name": i, "id": i} for i in df.columns]  #creates column list based on dataframe columns#
    df_dict =  df.to_dict('records')                          #converts df to dictionary#
    output_list = [column_list,df_dict]                       #combines df_dict and column_list into output list#
    return output_list                                        #returns list of column names and data values#

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:36] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:34:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:35:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:35:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 16:35:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [25/Oct/2021 16:38:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [25/Oct/2021 16:38:33] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
import pandas as pd
input_list = [1,2,3,4,5,6]
options = ['odd', 'div3']

df = pd.DataFrame(input_list, columns = ['id'])

if 'odd' in options:
    df['odd'] = [i%2!=0 for i in input_list]
if 'div3' in options:
    df['div3'] = [i%3==0 for i in input_list]
    
df2 = pd.DataFrame(input_list, columns = ['id'])

df2['oddtest'] = df['odd']
df2['div3test'] = df['div3']

test_list = [6,5,4,3,2,1]

df2['testadder'] = test_list

df2.head(6)

In [ ]:
client = CAVEclient("flywire_fafb_production")
mat_vers = max(client.materialize.get_versions())
syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[720575940608690478,720575940626744169 
]})
syn_df.head()